<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference:

https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go

https://www.kaggle.com/alhalimi/tokenization-and-word-embedding-compatibility

In [2]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c quora-insincere-questions-classification

Saving kaggle.json to kaggle.json
 32% 5.00M/15.8M [00:00<00:00, 36.6MB/s]
100% 15.8M/15.8M [00:00<00:00, 77.5MB/s]
100% 5.94G/5.96G [01:06<00:00, 51.0MB/s]
100% 5.96G/5.96G [01:06<00:00, 95.8MB/s]
  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 136MB/s]
 80% 44.0M/54.9M [00:00<00:00, 113MB/s] 
100% 54.9M/54.9M [00:00<00:00, 139MB/s]


In [3]:
import gc

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

import tensorflow as tf

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [15]:
import time
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [4]:
import os

# input_path = '/kaggle/input/quora-insincere-questions-classification'
input_path = './'
train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')
embeddings_path = os.path.join(input_path, 'embeddings.zip')

In [19]:
if os.path.exists(train_path): train_df = pd.read_csv(train_path)
else: train_df = pd.read_csv(train_path + '.zip')

if os.path.exists(test_path): test_df = pd.read_csv(train_path)
else: test_df = pd.read_csv(train_path + '.zip')

print('train shape:', train_df.shape)
print(train_df.head())
print()
print('test shape:', test_df.shape)
print(test_df.head())

train shape: (1306122, 3)
                    qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0
3  000042bf85aa498cd78e  ...      0
4  0000455dfa3e01eae3af  ...      0

[5 rows x 3 columns]

test shape: (1306122, 3)
                    qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0
3  000042bf85aa498cd78e  ...      0
4  0000455dfa3e01eae3af  ...      0

[5 rows x 3 columns]


In [26]:
# split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

# some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

# fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

# Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [31]:
%%time
# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [35]:
%%time
# Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

CPU times: user 12.3 s, sys: 413 ms, total: 12.7 s
Wall time: 12.5 s


In [54]:
%%time
import zipfile

glove = 'glove.840B.300d/glove.840B.300d.txt'
wiki = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

with zipfile.ZipFile(embeddings_path) as embeddings_zip:
    print("Found embeddings as a zip file")
    glove_embeddings = dict(get_coefs(*o.decode('utf-8').split(" ")) for o in embeddings_zip.open(glove))
    wiki_embeddings = dict(get_coefs(*o.decode('utf-8').split(" ")) for o in embeddings_zip.open(wiki))

Found embeddings as a zip file
CPU times: user 5min 54s, sys: 9.96 s, total: 6min 4s
Wall time: 6min 3s


In [56]:
def get_emb_stats(embeddings_index):
    """
    Get stats of a given embeddings index
    """   

    # Put all embeddings in a numpy matrix
    all_embs= np.stack(embeddings_index.values())

    # Get embedding stats
    emb_mean = all_embs.mean()
    emb_std = all_embs.std()
    
    num_embs = all_embs.shape[0]
    
    emb_size = all_embs.shape[1]
    
    return emb_mean,emb_std, num_embs, emb_size